In [ ]:
import torch
import wandb
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer, setup_chat_format

In [ ]:
from huggingface_hub import login


login()

In [ ]:
wandb.login()

In [ ]:
run = wandb.init(
    project="authorship_identification",
    job_type="training",
    entity="mlds23_ai",
    anonymous="allow",
    tags=["llm"],
)

In [ ]:
base_model = "IlyaGusev/saiga_llama3_8b"
new_model = "saiga_llama3_8b-rus_authors"

In [ ]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)


model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "up_proj",
        "down_proj",
        "gate_proj",
        "k_proj",
        "q_proj",
        "v_proj",
        "o_proj",
    ],
)
model = get_peft_model(model, peft_config)

In [ ]:
dataset = load_dataset("csv", split="train", data_files="rus_authors.csv")

In [ ]:
dataset

In [ ]:
def format_chat_template(row):
    row_json = [
        {"role": "user", "content": row["text"]},
        {"role": "assistant", "content": row["target"]},
    ]
    row["text2"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row


dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text2",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

In [ ]:
trainer.train()

In [ ]:
wandb.finish()
model.config.use_cache = True

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Узкими горными тропинками , от одного дачного поселка до другого , пробиралась вдоль южного берега Крыма маленькая бродячая труппа . Впереди обыкновенно бежал , свесив набок длинный розовый язык , белый пудель Арто , остриженный наподобие льва . У перекрестков он останавливался и , махая хвостом , вопросительно оглядывался назад . По каким-то ему одному известным признакам он всегда безошибочно узнавал дорогу и , весело болтая мохнатыми ушами , кидался галопом вперед .",
    }
]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

In [ ]:
trainer.model.save_pretrained(new_model)
# trainer.model.push_to_hub(new_model, use_temp_dir=False)